## TIM NEP CALCULATION
JPF 09Oct2020

What it does:
 - Uses Isaac's POP spreadsheet to find spills for specific pixels
 - Incorporates mirror emissivity and aperture elements
 - Uses Isaac's FWHM bandwidth estimates for these pixels
 - Assumes each pixel further has a wide band that terminates within the optics box (Toptics)
 - Computes NEP and power budgets for only these pixels (generally array edges)
 - Optics temperatures and emissivities are defined together in block 5 below
 - Complements other versions that makes nice spectral plots, but can't track variation in spill, resolution
 
 
 Things that aren't great:
 - Too much clumsy code repetition (LW/SW, stage to stage)
 - The data structure that comes out of the Excel read is clumsy
 - Isaac tells me that a lot of the power spill from the warm optics will eventually end up at the sky?

In [1]:
# packages
import os
import numpy as _np
import scipy.constants as _con
from scipy.integrate import quad
from scipy.integrate import trapz
import pandas as _pd
import matplotlib.pyplot as _plt

# constants
k = _con.k  # Boltzmann constant (J/K)
c = _con.c  # Speed of light (m/s)
h = _con.h  # Planck's Constant (J-s)

In [2]:
# physics utility functions
def n_occ (T,v):
    """Photon mode occupation number of a black body of temperature T at frequency v"""
    return 1./(_np.exp(h*v/(k*T))-1)

def IBlackBody (T, v, pol=False):
    """Planck's Law spectral radiance - radiated power (in the normal direction) per steradian
    per square meter per Hertz at frequency v - for a blackbody at temperature T.
    If pol=True, returns for one polarization (half the power of the standard expression)."""
    polfactor = 1 if pol else 2
    return polfactor * h/(c**2)*(v**3)*n_occ(T,v)

def PBBSingleModed (T, v1=0, v2=_np.inf, pol=False):
    """Planck's Law radiative power received by an ideal single-moded
    receiver sensitive over frequency band [v1,v2] observing a blackbody at temperature T.
    If pol=True, returns for one polarization (half the power of the standard expression)."""
    (out,err) = quad(lambda v: IBlackBody(T,v,pol=pol)*(c/v)**2, v1, v2)
    return out

def NEP_photon (T,eta,v1=0,v2=_np.inf, pol=False, narrow=False):
    """Photon NEP [W/rtHz] for a noiseless detector observing temperature T
    with quantum efficiency eta over a frequency band [v1,v2].
    Defined with an integral to hander wider bands; faster with narrow=True."""
    polfactor = 1 if pol else 2 # sum incoherently over two polarization modes
    if narrow:
        v0 = (v1+v2)/2.
        dv = (v2-v1)
        return _np.sqrt(2*polfactor)*h*v0*_np.sqrt(dv*eta*n_occ(T,v0)*(1+eta*n_occ(T,v0)))
    else:
        integrand = lambda v: eta*(h*v)**2*n_occ(T,v) + (eta*h*v*n_occ(T,v))**2
        # integrate and incorporate the factor of 2 from NEP definition
        return _np.sqrt(2*polfactor*quad(integrand,v1,v2)[0])

In [3]:
class amModel:
# Encapsulates an am atmosphere model
# Columns: f (GHz), tx , Tb (K), I (watt*cm-2*GHz-1*sr-1)
# tx: transmittance
# Tb: Planck brightness temperature (redundant with I)
# I: Spectral radiance (unpolarized)
    def __init__(self, filename, label=None):
        '''Initialize a new am model (code assumes we know the units!!!)'''
        self.atmo = _pd.read_csv(filename,header = None, sep=' ', names=['vGHz','tx','Tb','I'])
        self.atmo['n'] = self.atmo['I']*1e-5*c**2/(2*h*(self.atmo['vGHz']*1e9)**3)
        self.label=label
    def I(self,v):
        '''Atmospheric intensity as a function of frequency (W/m2/Hz/sr, dual-pol)'''
        return _np.interp(v/1e9,self.atmo['vGHz'],atmo['I'])*1e-5 # intensity in W/m2/Hz/sr (dual-pol)
    def n(self,v):
        '''Mode occupation number '''
        return self.I(v)*c**2/(2*h*v**3) # mode occupation number as a function of frequency
    def P(self,v1,v2,pol=False):
        """Returns the unobstructed single-moded power from the atmosphere model in a given band"""
        # implemented numerically, assuming that the data file is sampled much more finely than we need
        polfactor = 2 if pol else 1 # Iatm is unpolarized, so we divide by 2 for single-pol
        temp = self.atmo[(self.atmo['vGHz']>=v1/1e9) & (self.atmo['vGHz']<v2/1e9)]
        return trapz(temp['I']*1e-5 * (c/(temp['vGHz']*1e9))**2,x=temp['vGHz']*1e9)/polfactor
    def NEP(self,eta,v1,v2,pol=False):
        """Atmospheric NEP associated with a given band observed with efficiency eta"""
        # implemented numerically, assuming that the data file is sampled much more finely than we need
        polfactor = 1 if pol else 2 # sum incoherently over two polarization modes
        temp = self.atmo[(self.atmo['vGHz']>=v1/1e9) & (self.atmo['vGHz']<v2/1e9)]
        return _np.sqrt( polfactor * trapz( 
            (h*temp['vGHz']*1e9)**2*eta*temp['n'] + (eta*h*temp['vGHz']*1e9*temp['n'])**2,
            x=temp['vGHz']*1e9))

In [4]:
class POPanalysis:
    # Encapsulates Isaac's POP analysis
    # should make this more clever about identifying file features, but nah
    def __init__(self,filename):
        assert os.path.isfile(filename)
        self.filename = filename
    
    def parsePOPsheet(self,sheetname):
        # first, read in the whole sheet
        xls = _pd.read_excel(self.filename,sheet_name=sheetname,header=None)
        # construct a DataFrame with entries for each pixel
        df = _pd.DataFrame(columns=['Location','Wavelength','Spill'])
        start_row = 0
        this_entry = 0
        while xls.iloc[start_row,0][0]=='(': # new pixel!
            px = xls.iloc[start_row,0] # pixel identifier
            wvl = xls.iloc[start_row,1] # wavelength (um)
            x = _pd.DataFrame(data=_np.array(xls.iloc[(start_row+2):(start_row+9),1:6].copy()),index=xls.iloc[(start_row+2):(start_row+9),0].copy(),
                            columns=['TotalEff','Spill','RuzeLoss','Loss','Eff'])
            # tweak some things
            x['TotalEff']/=100.
            x['Spill']/=100.
            x['RuzeLoss']/=100.
            x['Loss']/=100.
            # make an entry
            df.loc[this_entry]={'Location':px,'Wavelength':wvl,'Spill':x.copy()}
            start_row += 10
            this_entry +=1
        return df

In [5]:
# ===== define the warm and cold optics =====

# Warm primary mirror, visible only through grating
Tprimary = 250. # Kelvin
epsPrimary = 0.005 # Emissivity of the primary
# Window
Twin = 250. # Kelvin
epsWin = 0.005 # Emissivity
# Filters
Tvcs2 = 175. # Kelvin
epsVCS2 = 0.005
Tvcs1 = 65. # Kelvin
epsVCS1 = 0.01
rVCS = 0.1 # Total reflection back into cold box
# Cold optics box, assumed visible across full band of each camera
Toptics = 4.9 # Kelvin
#Toptics = 3.4 # Kelvin
#Toptics = 6 # Kelvin
epsOptics = 0.005 # Emissivity of the cold reflectors

# Detector
detEff = 0.95 # efficiency of the detector itself
pol=False # polarization-insensitive
# Wide bandwidth of detector
SWlambda = _np.array([240e-6,317e-6]) # meters
LWlambda = _np.array([317e-6,420e-6]) # meters
SWnu = c/SWlambda[::-1] # Hz
LWnu = c/LWlambda[::-1] # Hz

# Atmosphere
atmo = amModel('Antarctic_DJF_37km_0_2000_10M_45_1.out',label='37km')

In [6]:
# load Isaac's optics POP analysis
POP = POPanalysis('../optics_analysis/ReverseModel_POP_Analysis.xlsx')
LWpop = POP.parsePOPsheet('LW 3mm Slit R_335 Spill')
SWpop = POP.parsePOPsheet('SW 2mm Slit R_435 Spill')
# add in by hand Isaac's estimates of bandwidth
# assume symmetric spatially
SWpop['FWHM'] = [1.43,1.50,1.43,1.70,1.29,1.70,1.97,1.43,1.97,1.29,1.36,1.50] # microns
# totally fake the LW numbers for now
LWpop['FWHM'] = LWpop['Wavelength']/155.0 # scaled mean resolving power above by spectrometer summary R ratio
# put in frequency units (Hz)
SWpop['v'] = c/(SWpop['Wavelength']*1e-6)
SWpop['dv'] = SWpop.v * (SWpop.FWHM/SWpop.Wavelength)
LWpop['v'] = c/(LWpop['Wavelength']*1e-6)
LWpop['dv'] = LWpop.v * (LWpop.FWHM/LWpop.Wavelength)

# DELETE FINAL ROWS FROM ISAAC'S SW SHEET, WHICH ARE BAD!!
SWpop=SWpop.drop([9,10,11],axis=0)

In [7]:
# summary structures: rows are pixels, columns are powers and such
bandsSW = _pd.DataFrame(data=SWpop.Location,columns=['Location'])
bandsLW = _pd.DataFrame(data=LWpop.Location,columns=['Location'])
for bands,pop in zip([bandsSW,bandsLW],[SWpop,LWpop]):
    # band definition
    bands['lam'] = pop.Wavelength
    bands['v'] = pop.v
    bands['dv'] = pop.dv

In [8]:
# optical loading model

# compute the wide-band loading separately
PWideLW = detEff*PBBSingleModed(Toptics,v1=LWnu[0],v2=LWnu[1],pol=pol)
PWideSW = detEff*PBBSingleModed(Toptics,v1=SWnu[0],v2=SWnu[1],pol=pol)

# in-band power from each element
# each element we spill some power and have losses on a few mirrors
# Aperture stages and reflector loss are added to Isaac's sims after-market, so we implement them differently

for bands,pop in zip([bandsSW,bandsLW],[SWpop,LWpop]): # cycle over cameras
    # need to create the columns first b/c the indexing is complex
    prefixes = ['P','Eff','NEP']
    stages = ['Grating','Slit','Lyot','VCS1','VCS2','Window','Secondary','Primary','Obscuration','Spider','Sky']
    for s in stages:
        for p in prefixes:
            bands[p+s]=0
    for idx in range(len(bands.index)): # cycle over pixels
        # preliminaries
        v1 = bands.loc[idx].v - bands.loc[idx].dv/2
        v2 = bands.loc[idx].v + bands.loc[idx].dv/2
        
        # grating has spill plus two reflective surfaces, all goes cold
        loss = pop.loc[idx].Spill.Spill.Grating + 2*epsOptics
        bands.loc[idx,'PGrating'] = detEff*loss*PBBSingleModed(Toptics, v1=v1, v2=v2, pol=pol)
        bands.loc[idx,'EffGrating'] = (1-loss)*detEff
        bands.loc[idx,'NEPGrating'] = NEP_photon(Toptics,loss*detEff,v1=v1,v2=v2, pol=pol, narrow=True)
        
        # slit has spill plus four reflective surfaces, all goes cold
        # here and below, spill in the table has been re-normalized to total, so we need to undo that
        spill = pop.loc[idx].Spill.Spill.Slit / pop.loc[idx].Spill.TotalEff.Grating
        loss = spill + 4*epsOptics
        bands.loc[idx,'PSlit'] = bands.loc[idx,'EffGrating']*loss*PBBSingleModed(Toptics, v1=v1, v2=v2, pol=pol)
        bands.loc[idx,'EffSlit'] = (1-loss)*bands.loc[idx,'EffGrating']
        bands.loc[idx,'NEPSlit'] = NEP_photon(Toptics,loss*bands.loc[idx,'EffGrating'],v1=v1,v2=v2, pol=pol, narrow=True)
        
        # Lyot has spill plus five reflective surfaces, all goes cold
        spill = pop.loc[idx].Spill.Spill.Lyot / pop.loc[idx].Spill.TotalEff.Slit
        loss = spill + 5*epsOptics
        bands.loc[idx,'PLyot'] = bands.loc[idx,'EffSlit']*loss*PBBSingleModed(Toptics, v1=v1, v2=v2, pol=pol)
        bands.loc[idx,'EffLyot'] = (1-loss)*bands.loc[idx,'EffSlit']
        bands.loc[idx,'NEPLyot'] = NEP_photon(Toptics,loss*bands.loc[idx,'EffSlit'],v1=v1,v2=v2, pol=pol, narrow=True)
        
        # VCS1
        refl = rVCS # reflected cold
        loss = epsVCS1 # lost at this temperature
        bands.loc[idx,'PVCS1'] = bands.loc[idx,'EffLyot']*loss*PBBSingleModed(Tvcs1, v1=v1, v2=v2, pol=pol)+ bands.loc[idx,'EffLyot']*refl*PBBSingleModed(Toptics, v1=v1, v2=v2, pol=pol)
        bands.loc[idx,'EffVCS1'] = (1-refl)*(1-loss)*bands.loc[idx,'EffLyot']
        bands.loc[idx,'NEPVCS1'] = NEP_photon(Tvcs1,loss*bands.loc[idx,'EffLyot'],v1=v1,v2=v2, pol=pol, narrow=True)
        # VCS2
        loss = epsVCS2 # lost at this temperature
        bands.loc[idx,'PVCS2'] = bands.loc[idx,'EffVCS1']*loss*PBBSingleModed(Tvcs2, v1=v1, v2=v2, pol=pol)
        bands.loc[idx,'EffVCS2'] = (1-loss)*bands.loc[idx,'EffVCS1']
        bands.loc[idx,'NEPVCS2'] = NEP_photon(Tvcs2,loss*bands.loc[idx,'EffVCS1'],v1=v1,v2=v2, pol=pol, narrow=True)
        # Window
        loss = epsWin # lost at this temperature
        bands.loc[idx,'PWindow'] = bands.loc[idx,'EffVCS2']*loss*PBBSingleModed(Twin, v1=v1, v2=v2, pol=pol)
        bands.loc[idx,'EffWindow'] = (1-loss)*bands.loc[idx,'EffVCS2']
        bands.loc[idx,'NEPWindow'] = NEP_photon(Twin,loss*bands.loc[idx,'EffVCS2'],v1=v1,v2=v2, pol=pol, narrow=True)

        # Secondary - spill to atmosphere, emissivity to ambient
        spill = pop.loc[idx].Spill.Spill.Secondary / pop.loc[idx].Spill.TotalEff.Lyot
        loss = epsPrimary
        bands.loc[idx,'PSecondary'] = bands.loc[idx,'EffWindow']*loss*PBBSingleModed(Tprimary, v1=v1, v2=v2, pol=pol) + bands.loc[idx,'EffWindow']*spill*atmo.P(v1=v1, v2=v2, pol=pol)
        bands.loc[idx,'EffSecondary'] = (1-loss-spill)*bands.loc[idx,'EffWindow']
        bands.loc[idx,'NEPSecondary'] = NEP_photon(Tprimary,loss*bands.loc[idx,'EffWindow'],v1=v1,v2=v2, pol=pol, narrow=True)

        # Primary
        spill = pop.loc[idx].Spill.Spill.Primary / pop.loc[idx].Spill.TotalEff.Secondary
        loss = spill + epsPrimary
        bands.loc[idx,'PPrimary'] = bands.loc[idx,'EffSecondary']*loss*PBBSingleModed(Tprimary, v1=v1, v2=v2, pol=pol)
        bands.loc[idx,'EffPrimary'] = (1-loss)*bands.loc[idx,'EffSecondary']
        bands.loc[idx,'NEPPrimary'] = NEP_photon(Tprimary,loss*bands.loc[idx,'EffSecondary'],v1=v1,v2=v2, pol=pol, narrow=True)
        
        # Obscuration
        spill = pop.loc[idx].Spill.Spill.Obscuration / pop.loc[idx].Spill.TotalEff.Primary
        loss = spill + epsPrimary
        bands.loc[idx,'PObscuration'] = bands.loc[idx,'EffPrimary']*loss*PBBSingleModed(Tprimary, v1=v1, v2=v2, pol=pol)
        bands.loc[idx,'EffObscuration'] = (1-loss)*bands.loc[idx,'EffPrimary']
        bands.loc[idx,'NEPObscuration'] = NEP_photon(Tprimary,loss*bands.loc[idx,'EffPrimary'],v1=v1,v2=v2, pol=pol, narrow=True)
        
        # Spider
        spill = pop.loc[idx].Spill.Spill.Spider / pop.loc[idx].Spill.TotalEff.Obscuration
        loss = spill + epsPrimary
        bands.loc[idx,'PSpider'] = bands.loc[idx,'EffObscuration']*loss*PBBSingleModed(Tprimary, v1=v1, v2=v2, pol=pol)
        bands.loc[idx,'EffSpider'] = (1-loss)*bands.loc[idx,'EffObscuration']
        bands.loc[idx,'NEPSpider'] = NEP_photon(Tprimary,loss*bands.loc[idx,'EffObscuration'],v1=v1,v2=v2, pol=pol, narrow=True)
        
        # Atmosphere
        bands.loc[idx,'PSky'] = bands.loc[idx,'EffSpider']*(PBBSingleModed(2.73, v1=v1, v2=v2, pol=pol) + atmo.P(v1=v1, v2=v2, pol=pol))
        bands.loc[idx,'EffSky'] = bands.loc[idx,'EffSpider']
        bands.loc[idx,'NEPSky'] = atmo.NEP(bands.loc[idx,'EffSky'],v1,v2,pol=pol)


In [9]:
# Make a POWER summary table
powerSW = _pd.concat((bandsSW[['Location','lam','EffSky']].copy(), bandsSW.filter(regex='^P.+')), axis=1)
powerSW['PBox'] = PWideSW
powerSW['PTOTAL'] = powerSW.filter(regex='^P.+').sum(axis=1)
for col in powerSW.filter(regex='^P.+').columns:
    powerSW[col] = (powerSW[col]*1e12).round(2)
powerSW['EffSky'] = powerSW['EffSky'].round(3)

powerLW = _pd.concat((bandsLW[['Location','lam','EffSky']].copy(), bandsLW.filter(regex='^P.+')), axis=1)
powerLW['PBox'] = PWideLW
powerLW['PTOTAL'] = powerLW.filter(regex='^P.+').sum(axis=1)# DISPLAY
for col in powerLW.filter(regex='^P.+').columns:
    powerLW[col] = (powerLW[col]*1e12).round(2)
powerLW['EffSky'] = powerLW['EffSky'].round(3)

In [10]:
# Print compact POWER summary
powerSWsm = powerSW[['Location','lam','EffSky','PBox']].copy()
powerSWsm['Pcold'] = powerSW[['PGrating','PSlit','PLyot']].sum(axis=1)
powerSWsm = _pd.concat((powerSWsm,powerSW[['PVCS1','PVCS2','PWindow']]), axis=1)
powerSWsm['Pwarm'] = powerSW[['PSecondary','PPrimary','PObscuration','PSpider']].sum(axis=1)
powerSWsm = _pd.concat((powerSWsm,powerSW[['PSky','PTOTAL']]), axis=1)
print('SW Camera [pW]')
powerSWsm

SW Camera [pW]


,Location,lam,EffSky,PBox,Pcold,PVCS1,PVCS2,PWindow,Pwarm,PSky,PTOTAL
0,"(73, -64)",235.60,0.421,0.01,0.0,0.05,0.08,0.11,1.67,0.04,1.96
1,"(73, 0)",237.25,0.475,0.01,0.0,0.05,0.08,0.13,1.57,0.04,1.88
2,"(73, 64)",235.60,0.421,0.01,0.0,0.05,0.08,0.11,1.67,0.04,1.96
3,"(0, -64)",279.50,0.370,0.01,0.0,0.04,0.06,0.09,1.49,0.04,1.73
4,"(0, 0)",281.40,0.412,0.01,0.0,0.03,0.05,0.07,0.77,0.00,0.93
5,"(0, 64)",279.50,0.369,0.01,0.0,0.04,0.06,0.09,1.48,0.04,1.72
6,"(-73, -64)",318.40,0.341,0.01,0.0,0.03,0.05,0.07,1.26,0.00,1.44
7,"(-73, 0)",320.75,0.344,0.01,0.0,0.02,0.03,0.05,0.61,0.04,0.76
8,"(-73, 64)",318.40,0.340,0.01,0.0,0.03,0.05,0.07,1.26,0.00,1.43


In [11]:
#Compact power summary
powerLWsm = powerLW[['Location','lam','EffSky','PBox']].copy()
powerLWsm['Pcold'] = powerLW[['PGrating','PSlit','PLyot']].sum(axis=1)
powerLWsm = _pd.concat((powerLWsm,powerLW[['PVCS1','PVCS2','PWindow']]), axis=1)
powerLWsm['Pwarm'] = powerLW[['PSecondary','PPrimary','PObscuration','PSpider']].sum(axis=1)
powerLWsm = _pd.concat((powerLWsm,powerLW[['PSky','PTOTAL']]), axis=1)
print('LW Camera [pW]')
powerLWsm

LW Camera [pW]


,Location,lam,EffSky,PBox,Pcold,PVCS1,PVCS2,PWindow,Pwarm,PSky,PTOTAL
0,"(-73, -50.75)",312.75,0.473,0.08,0.0,0.05,0.07,0.10,1.40,0.05,1.74
1,"(-73, 0)",314.00,0.486,0.08,0.0,0.05,0.07,0.11,1.41,0.02,1.73
2,"(-73, 50.75)",312.75,0.475,0.08,0.0,0.05,0.07,0.10,1.41,0.05,1.75
3,"(0, -50.75)",370.50,0.424,0.08,0.0,0.04,0.06,0.08,1.05,0.04,1.36
4,"(0, 0)",371.50,0.406,0.08,0.0,0.04,0.05,0.08,1.04,0.02,1.31
5,"(0, 50.75)",370.50,0.428,0.08,0.0,0.04,0.06,0.08,1.08,0.04,1.38
6,"(73, -50.75)",422.50,0.371,0.08,0.0,0.03,0.04,0.06,0.82,0.01,1.05
7,"(73, 0)",423.90,0.345,0.08,0.0,0.03,0.04,0.06,0.70,0.00,0.91
8,"(73, 50.75)",422.50,0.371,0.08,0.0,0.03,0.04,0.06,0.83,0.01,1.05


In [12]:
# Make a NOISE summary table
noiseSW = _pd.concat((bandsSW[['Location','lam','EffSky']].copy(), bandsSW.filter(regex='^NEP.+')), axis=1)
noiseSW['NEPBox']= bandsSW.apply(lambda row: NEP_photon(Toptics,detEff,v1=SWnu[0],v2=SWnu[1],pol=pol, narrow=False),axis=1)
noiseSW['NEPCold'] = _np.sqrt(noiseSW['NEPGrating']**2 + noiseSW['NEPSlit']**2 + noiseSW['NEPLyot']**2)
noiseSW['NEPWarm'] = _np.sqrt(noiseSW['NEPSecondary']**2 + noiseSW['NEPPrimary']**2 + noiseSW['NEPObscuration']**2 + noiseSW['NEPSpider']**2)
noiseSW['NEPTotal'] = _np.sqrt(noiseSW['NEPBox']**2 + noiseSW['NEPCold']**2 + noiseSW['NEPVCS1']**2 + noiseSW['NEPVCS2']**2 + noiseSW['NEPWindow']**2 + noiseSW['NEPWarm']**2 + noiseSW['NEPSky']**2)
for col in noiseSW.filter(regex='^NEP.+').columns:
    noiseSW[col] = (noiseSW[col]*1e18).round(2)
noiseSW['EffSky'] = noiseSW['EffSky'].round(3)

noiseLW = _pd.concat((bandsLW[['Location','lam','EffSky']].copy(), bandsLW.filter(regex='^NEP.+')), axis=1)
noiseLW['NEPBox']= bandsLW.apply(lambda row: NEP_photon(Toptics,detEff,v1=LWnu[0],v2=LWnu[0],pol=pol, narrow=False),axis=1)
noiseLW['NEPBox']= bandsLW.apply(lambda row: NEP_photon(Toptics,detEff,v1=row.v-row.dv/2,v2=row.v+row.dv/2,pol=pol, narrow=False),axis=1)
noiseLW['NEPCold'] = _np.sqrt(noiseLW['NEPGrating']**2 + noiseLW['NEPSlit']**2 + noiseLW['NEPLyot']**2)
noiseLW['NEPWarm'] = _np.sqrt(noiseLW['NEPSecondary']**2 + noiseLW['NEPPrimary']**2 + noiseLW['NEPObscuration']**2 + noiseLW['NEPSpider']**2)
noiseLW['NEPTotal'] = _np.sqrt(noiseLW['NEPBox']**2 + noiseLW['NEPCold']**2 + noiseLW['NEPVCS1']**2 + noiseLW['NEPVCS2']**2 + noiseLW['NEPWindow']**2 + noiseLW['NEPWarm']**2 + noiseLW['NEPSky']**2)
for col in noiseLW.filter(regex='^NEP.+').columns:
    noiseLW[col] = (noiseLW[col]*1e18).round(2)
noiseLW['EffSky'] = noiseLW['EffSky'].round(3)

In [13]:
# Print compact NOISE summary
cols = ['Location','lam','EffSky','NEPBox','NEPCold','NEPVCS1','NEPVCS2','NEPWindow','NEPWarm','NEPTotal']
print('SW Camera Noise [aW/rtHz]')
noiseSW[cols]

SW Camera Noise [aW/rtHz]


,Location,lam,EffSky,NEPBox,NEPCold,NEPVCS1,NEPVCS2,NEPWindow,NEPWarm,NEPTotal
0,"(73, -64)",235.60,0.421,4.05,0.18,8.82,11.38,13.97,54.43,58.53
1,"(73, 0)",237.25,0.475,4.05,0.19,9.25,11.93,14.64,52.77,57.30
2,"(73, 64)",235.60,0.421,4.05,0.18,8.82,11.39,13.98,54.49,58.59
3,"(0, -64)",279.50,0.370,4.05,0.41,7.31,9.18,11.21,47.60,50.79
4,"(0, 0)",281.40,0.412,4.05,0.35,6.48,8.13,9.93,33.93,37.09
5,"(0, 64)",279.50,0.369,4.05,0.41,7.29,9.16,11.19,47.43,50.62
6,"(-73, -64)",318.40,0.341,4.05,0.67,6.43,7.93,9.66,41.05,43.60
7,"(-73, 0)",320.75,0.344,4.05,0.59,5.34,6.59,8.02,28.04,31.14
8,"(-73, 64)",318.40,0.340,4.05,0.67,6.42,7.92,9.65,40.95,43.50


In [14]:
print('LW Camera Noise [aW/rtHz]')
noiseLW[cols]

LW Camera Noise [aW/rtHz]


,Location,lam,EffSky,NEPBox,NEPCold,NEPVCS1,NEPVCS2,NEPWindow,NEPWarm,NEPTotal
0,"(-73, -50.75)",312.75,0.473,0.89,0.54,7.67,9.49,11.58,43.72,47.34
1,"(-73, 0)",314.00,0.486,0.90,0.54,7.71,9.54,11.63,44.21,47.48
2,"(-73, 50.75)",312.75,0.475,0.89,0.54,7.69,9.51,11.60,43.84,47.46
3,"(0, -50.75)",370.50,0.424,1.44,0.93,6.38,7.74,9.41,35.00,38.01
4,"(0, 0)",371.50,0.406,1.45,0.97,6.21,7.53,9.16,35.11,37.81
5,"(0, 50.75)",370.50,0.428,1.44,0.92,6.42,7.78,9.46,35.31,38.33
6,"(73, -50.75)",422.50,0.371,1.92,1.35,5.35,6.40,7.77,28.89,31.22
7,"(73, 0)",423.90,0.345,1.93,1.41,5.14,6.15,7.45,26.48,28.78
8,"(73, 50.75)",422.50,0.371,1.92,1.34,5.36,6.41,7.78,28.93,31.25
